In [24]:
#importing important modules & libraries
from links import get_houses_info, find_all_links
from concurrent.futures import ThreadPoolExecutor, ProcessPoolExecutor, as_completed
from itertools import chain
import time
import pandas as pd
pd.options.display.max_columns = None
pd.options.display.max_colwidth = None
import requests
from bs4 import BeautifulSoup
import re
from datetime import datetime
import numpy as np

In [26]:
#Threads
with ThreadPoolExecutor(max_workers=20) as threads:
    links = threads.map(find_all_links, range(100))
    links = list(chain.from_iterable(links))

IndexError: list index out of range

In [ ]:
#Processes
with ProcessPoolExecutor(max_workers=25) as processes:
    data1 = processes.map(get_houses_info, links)
    data1 = list(data1)

In [ ]:
#Converting the returned info into a dataframe
houses = pd.DataFrame(data1, columns = ['Date','Link','Price','Location','Details'])
houses.head()

,Date,Link,Price,Location,Details


In [ ]:
##################################Extracting the number of bedrooms#########################
def extract_bedrooms(text):
    """Extracts the number of bedrooms written in different variations"""
    
    bedrooms1 = r'(\d+)-?\s*-?(?:bed\s?rooms?|rooms?)'
    bedrooms2 = r'(?:ibyumba|chambres?)\s*(\d)'
    
    b1 = re.findall(bedrooms1, text, re.I) 
    
    if len(b1) > 0:
        rooms = b1[0]
    else:
        b2 = re.findall(bedrooms2, text, re.I)
        if len(b2) > 0:
            rooms = b2[0]
        else:
            rooms = np.nan
    
    return rooms
###########################Extracting the number of bathrooms###############################################################
def extract_bathrooms(text):
    bathrooms1 = r'(\d+)-?\s*-?(?:bathrooms?|restrooms?|bath|bothrooms?)'
    bathrooms2 = r'(?:toilets?|douches?|toilettes?|tuwareti|tuwarete|ubwiherero)\s*(\d)'
    b1 = re.findall(bathrooms1, text, re.I) 
    
    if len(b1) > 0:
        bathrooms = b1[0]
    else:
        b2 = re.findall(bathrooms2, text, re.I)
        if len(b2) > 0:
            bathrooms = b2[0]
        else:
            bathrooms3 = r'(toilets?|douches?|toilettes?|tuwareti|tuwarete|ubwiherero|bathrooms?|bath)'
            b3 = re.findall(bathrooms3, text, re.I)
            if len(b3) > 0:
                bathrooms = '1'
            else:
                bathrooms = np.nan
    
    return bathrooms
##############################################Extracting the plotsize#####################################################
def extract_plotsize(text):
    plotsize1 = r'(?:spaceplotsize|plotsize|plot size|plotize).*?(\d+.{0,2}\d*.)'
    plotsize2 = r'(\b\d{,2}/\d{,2}\b)'
    plotsize3 = r'(\d+\s*sqm)'
    plotsize4 = r'(sqm\s*\d+)'
    
    p1 = re.findall(plotsize1, text, re.I)
    p2 = re.findall(plotsize2, text, re.I)
    p3 = re.findall(plotsize3, text, re.I)
    p4 = re.findall(plotsize4, text, re.I)
    
    if len(p1) > 0:
        size = p1[0]
    elif len(p2) > 0:
        size = p2[0]    
    elif len(p3) > 0:
        size = p3[0]
    elif len(p4) > 0:
        size = p4[0]
    else:
        size = np.nan
            
    return size
##################################Checking if has boys quarters or annexes#########################
def extract_quarters(text):
    boys = r'(boys?)'
    annexes = r'(Annex[es]?|anegisi)'
    maids = r'(maids?|servants?|keeper)'
    a = re.findall(annexes, text, re.I)
    b = re.findall(boys, text, re.I)
    m = re.findall(maids, text, re.I)
    
    if len(a) > 0:
        quarters = 'Yes'
    elif len(b) > 0:
        quarters = 'Yes'
    elif len(m) > 0:
        quarters = 'Yes'
    else:
        quarters = np.nan
    return quarters

In [ ]:
houses['Size'] = houses['Details'].apply(extract_plotsize)
houses['Bedrooms'] = houses['Details'].apply(extract_bedrooms)
houses['Bathrooms'] = houses['Details'].apply(extract_bathrooms)
houses['Quarters'] = houses['Details'].apply(extract_quarters)
houses['Wardrobes'] = houses['Details'].str.extract(r'(Wardrobes?)',re.I)
houses['Cabinets'] = houses['Details'].str.extract(r'(cabinets?)', re.I)
houses['Balcony'] = houses['Details'].str.extract(r'(Balcony)',re.I)
houses['Parking'] = houses['Details'].str.extract(r'(parking|pariking|parikingi)', re.I)

In [ ]:
houses['Parking'] = houses['Parking'].apply(lambda x: 'Yes' if str(x) != 'nan' else x)
houses['Balcony'] = houses['Balcony'].apply(lambda x: 'Yes' if str(x)!='nan' else x)
houses['Cabinets'] = houses['Cabinets'].apply(lambda x: 'Yes' if str(x)!='nan' else x)
houses['Wardrobes'] = houses['Wardrobes'].apply(lambda x: 'Yes' if str(x)!='nan' else x)

In [ ]:
houses

,Date,Link,Price,Location,Details,Size,Bedrooms,Bathrooms,Quarters,Wardrobes,Cabinets,Balcony,Parking


In [ ]:
#Saving the dataset to csv
houses.to_csv('nairobi houses.csv', index = False)